# Colour Correction using Splines
## Exposure Invariance

### Define constants

In [ ]:
# Enable automatic reloading of modules before executing code to ensure latest changes are used.
%load_ext autoreload
%autoreload 2

# Importing necessary libraries and functions.
# 'colour' is used for color science computations.
# 'data' contains functions to load various datasets and color matching functions.
import colour
from data import (load_dataset_sfu, load_dataset_csv, load_cmfs, 
                  load_camera, load_insitu, msds_to_rgb, msds_to_xyz, load_dataset_skin)
import numpy as np

# Setting a seed for random number generation to ensure reproducibility.
RANDOM_STATE = 0
np.random.seed(RANDOM_STATE)

# Defining file paths for the datasets. These files should contain the data needed for analysis.
SFU_FILE_PATH = 'data/reflect_db.reflect'  # SFU material reflectance database
CAVE_FOSTER2004_PATH = 'data/cave_foster2002.csv'  # CAVE dataset for color constancy
INSITU_PATH = "data/insitu_dataset.csv"  # In-situ measurements dataset
CAVE_PATH = 'data/cave.csv'  # Another CAVE dataset file
FOSTER_50_PATH = 'data/foster50.csv'  # Foster dataset with 50 images for color analysis
CAMERA = 'nikon'  # Specifying the camera model used in the datasets or for simulation


In [ ]:
cave_foster2004_dataset = load_dataset_csv(CAVE_FOSTER2004_PATH)
foster_50_dataset = load_dataset_csv(FOSTER_50_PATH)
insitu = load_insitu(INSITU_PATH)

sfu = load_dataset_sfu(SFU_FILE_PATH)


In [ ]:
TRAIN = insitu
TEST = sfu


### Computing Observer Responses
We can easily change the order of test and train sets here

In [ ]:
from plotting import plot_chromaticity_diagram
from colour.characterisation import training_data_sds_to_XYZ
from colour import XYZ_to_Lab
cmfs, illuminant = load_cmfs()
response_trainset_xyz = msds_to_xyz(TRAIN, cmfs, illuminant)
response_testset_xyz = msds_to_xyz(TEST, cmfs, illuminant) / 5

lab = XYZ_to_Lab(response_trainset_xyz)

### Computing Camera Responses

In [ ]:
import numpy as np
MSDS_TRAIN = load_camera(CAMERA)

response_trainset_camera = msds_to_rgb(TRAIN,MSDS_TRAIN, illuminant)
response_testset_camera = msds_to_rgb(TEST,MSDS_TRAIN, illuminant) / 5

In [137]:
# Scale 
from colour import delta_E
k = 4
response_testset_camera_scaled = response_testset_camera * k
response_testset_xyz_scaled = response_testset_xyz  * k


lab1 = XYZ_to_Lab(response_testset_xyz)
lab2 = XYZ_to_Lab(response_testset_xyz)

print(delta_E(lab1, lab2))

[ 0.  0.  0. ...,  0.  0.  0.]


### Fit Generalized Additive Model with P-splines

### Nikon

In [ ]:
from models import GAMOptimizer
from evaluate import pred, pred_ei
def test_gam_optimizer(splines_lams_dict, order_value=3):
    """
    Test GAMOptimizer models with different numbers of splines and corresponding lambda values.

    Parameters:
    - splines_lams_dict: Dictionary where keys are spline numbers and values are the corresponding lambda values for regularization.
    - order_value: The order of the spline. Default is 3.
    """
    for n_splines, lams_value in splines_lams_dict.items():
        print(f"Testing GAMOptimizer with {n_splines} splines and lambda {lams_value}")
        gam = GAMOptimizer(lams=lams_value, order=order_value, n_splines=n_splines)
        gam.fit(response_trainset_camera, response_trainset_xyz)
        pred_ei(gam, response_testset_camera_scaled, response_testset_xyz_scaled, f"DeltaE Foster+CAVE with {n_splines} splines and lambda {lams_value}", k)

# Example usage: Create a dictionary mapping spline numbers to their corresponding lambda values and pass it to the function.
splines_lams_dict = {
    5: 1e-6,
    10: 0.0001,
    20: 0.01
}
if CAMERA == 'nikon':
    test_gam_optimizer(splines_lams_dict)

### Sigma

In [ ]:
def test_gam_optimizer(splines_lams_dict, order_value=3):
    """
    Test GAMOptimizer models with different numbers of splines and corresponding lambda values.

    Parameters:
    - splines_lams_dict: Dictionary where keys are spline numbers and values are the corresponding lambda values for regularization.
    - order_value: The order of the spline. Default is 3.
    """
    for n_splines, lams_value in splines_lams_dict.items():
        print(f"Testing GAMOptimizer with {n_splines} splines and lambda {lams_value}")
        gam = GAMOptimizer(lams=lams_value, order=order_value, n_splines=n_splines)
        gam.fit(response_trainset_camera, response_trainset_xyz)
        pred(gam, response_testset_camera, response_testset_xyz, f"{n_splines} splines and lambda {lams_value}")

# Example usage: Create a dictionary mapping spline numbers to their corresponding lambda values and pass it to the function.
splines_lams_dict = {
    5: 1e-9,
    10: 0.0001,
    20: 0.0001
}
if CAMERA == 'sigma':
    test_gam_optimizer(splines_lams_dict)

In [ ]:
from models import RGBtoXYZNetwork

mcdonalds_nn = RGBtoXYZNetwork()
mcdonalds_nn.fit(response_trainset_camera, response_trainset_xyz)
#pred_ei(mcdonalds_nn, response_testset_camera, response_testset_xyz, "Foster 50")

### Fit Linear Model

In [133]:
from sklearn.linear_model import LinearRegression
from evaluate import pred, pred_ei

linear = LinearRegression(fit_intercept=False)


linear.fit(response_trainset_camera, response_trainset_xyz)
pred_ei(linear, response_testset_camera_scaled, response_testset_xyz_scaled, "Foster 50", k)


---- RESULTS Foster 50 ----
DeltaE mean: 0.86
DeltaE max: 4.89
DeltaE median: 0.62
DeltaE 95 percentile: 2.48
DeltaE 99 percentile: 3.70


array([[ 0.1502476 ,  0.17350496,  0.48297144],
       [ 0.06908994,  0.07958993,  0.25862355],
       [ 0.28081306,  0.2017619 ,  0.07543282],
       ..., 
       [ 0.11167944,  0.065479  ,  0.08521249],
       [ 0.04114221,  0.0421851 ,  0.078295  ],
       [ 0.25659453,  0.26900094,  0.29904101]])

In [ ]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000LCC = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=1))
])


DE2000LCC.fit(response_trainset_camera, response_trainset_xyz)
pred_ei(DE2000LCC, response_testset_camera_scaled, response_testset_xyz_scaled, "Foster 50", k)


### Fit 3rd order Root-Polynomial Model

In [134]:
from models import PolynomialTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])

RP_linear_3.fit(response_trainset_camera, response_trainset_xyz)

pred_ei(RP_linear_3, response_testset_camera_scaled, response_testset_xyz_scaled, "Foster 50", k)


---- RESULTS Foster 50 ----
DeltaE mean: 0.69
DeltaE max: 3.55
DeltaE median: 0.56
DeltaE 95 percentile: 1.79
DeltaE 99 percentile: 2.76


array([[ 0.13697115,  0.1651093 ,  0.45953189],
       [ 0.05691822,  0.07194525,  0.23673991],
       [ 0.27923484,  0.20050752,  0.07615461],
       ..., 
       [ 0.11764376,  0.06906851,  0.0944686 ],
       [ 0.04201113,  0.04272232,  0.07978572],
       [ 0.25732676,  0.26947671,  0.30009502]])

### Fit a 2nd order Root-Polynomial Model

In [ ]:
from models import GAMOptimizer, PolynomialTransformer, DeltaEOptimizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

RP_linear_2 = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=True)),
    ('regressor', LinearRegression(fit_intercept=False))
])


RP_linear_2.fit(response_trainset_camera, response_trainset_xyz)
pred_ei(RP_linear_2, response_testset_camera_scaled, response_testset_xyz_scaled, "Foster 50", k)


In [135]:
from colour import XYZ_to_Lab

P_Linear_3 = Pipeline([
    ('transformer', PolynomialTransformer(degree=3, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


P_Linear_3.fit(response_trainset_camera, response_trainset_xyz)
prediction = pred_ei(P_Linear_3, response_testset_camera_scaled, response_testset_xyz_scaled, "Foster 50", k)

---- RESULTS Foster 50 ----
DeltaE mean: 0.74
DeltaE max: 4.37
DeltaE median: 0.54
DeltaE 95 percentile: 2.11
DeltaE 99 percentile: 3.22


In [ ]:
PCC_2 = Pipeline([
    ('transformer', PolynomialTransformer(degree=2, rp=False)),
    ('regressor', LinearRegression(fit_intercept=False))
])


PCC_2.fit(response_trainset_camera, response_trainset_xyz)
pred_ei(PCC_2, response_testset_camera_scaled, response_testset_xyz_scaled, "Foster 50", k)


In [ ]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000P = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=False, degree=3))
])


DE2000P.fit(response_trainset_camera, response_trainset_xyz)
pred_ei(DE2000P, response_testset_camera_scaled, response_testset_xyz_scaled, "Foster 50", k)


In [ ]:
from sklearn.pipeline import Pipeline
from models import DeltaEOptimizer

DE2000RP = Pipeline([
    ('regressor', DeltaEOptimizer(root_polynomial=True, degree=3))
])


DE2000RP.fit(response_trainset_camera, response_trainset_xyz)
pred_ei(DE2000RP, response_testset_camera_scaled, response_testset_xyz_scaled, "Foster 50", k)
